In [1]:
import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup as bs
import csv

In [2]:
URL = "https://www.imdb.com/chart/toptv/"
r = requests.get(URL)
  
soup = bs(r.content, "html.parser")
#print(soup.prettify())

In [3]:
table = soup.find("table", attrs={"class":"chart full-width"})

table_body = table.find("tbody")


In [4]:
%%time
imdb_movie = []
rank = 1

for i in table_body.findAll("tr"):
    imdb = {}
    genre_list = []
    for j in i.findAll("td"):
        imdb["Rank"] = rank
        if j.get("class")[0]=="titleColumn":
            print(j.a.text)
            imdb["Title"] = j.a.text
            movie_url = "https://www.imdb.com/"+j.a["href"]
            r = requests.get(movie_url)
            soup = bs(r.content, "html.parser")
            imdb["duration"] = soup.find("div",attrs={"class":"sc-80d4314-2"})\
                          .findAll("li",attrs={"class":"ipc-inline-list__item"})[-1].text
            a_list = soup.find("div",attrs={"class":"ipc-chip-list__scroller"}).findAll("a")
            for genres in a_list:
                genre_list.append(genres.text)
            imdb["genre"] = genre_list
        elif "imdbRating" in j.get("class"):
            imdb["Rating"] = j.strong.text[:4]
        if j.find("span",attrs={"class":"secondaryInfo"})!=None:
            imdb['year'] = j.span.text.replace("(","").replace(")","")
    rank += 1
    imdb_movie.append(imdb)
    
col = imdb.keys()

Planet Earth II
Breaking Bad
Planet Earth
Band of Brothers
Chernobyl
The Wire
Blue Planet II
Avatar: The Last Airbender
Cosmos: A Spacetime Odyssey
The Sopranos
Cosmos
Our Planet
Game of Thrones
Rick and Morty
The World at War
Hagane no renkinjutsushi
The Last Dance
Life
Sherlock
The Twilight Zone
The Vietnam War
Batman: The Animated Series
Scam 1992: The Harshad Mehta Story
Arcane: League of Legends
The Blue Planet
Shingeki no Kyojin
The Office
Firefly
Human Planet
Frozen Planet
Death Note: Desu nôto
Only Fools and Horses....
Hunter x Hunter
The Civil War
True Detective
Seinfeld
The Beatles: Get Back
Sahsiyet
Better Call Saul
Dekalog
Fargo
Clarkson's Farm
Kaubôi bibappu
Nathan for You
Gravity Falls
When They See Us
Last Week Tonight with John Oliver
Friends
TVF Pitchers
Africa
Apocalypse: La 2ème guerre mondiale
Monty Python's Flying Circus
It's Always Sunny in Philadelphia
Taskmaster
Das Boot
Curb Your Enthusiasm
The West Wing
Pride and Prejudice
Leyla ile Mecnun
Fawlty Towers
Freaks

In [5]:
filename = 'imdb_top_250_shows.csv'
with open(filename, 'w', newline='') as f:
    w = csv.DictWriter(f,col)
    w.writeheader()
    for imdb in imdb_movie:
        w.writerow(imdb)